datetime	日時（YYYY-MM-DD）	文字列
y	引越し数	数値：整数
client	法人が絡む特殊な引越し日フラグ	数値：整数（0,1）
close	休業日	数値：整数（0,1）
price_am	午前の料金区分（-1は欠損を表す。5が最も料金が高い）	数値：整数（-1,0,1,2,3,4,5）
price_pm	午後の料金区分（-1は欠損を表す。5が最も料金が高い）	数値：整数（-1,0,1,2,3,4,5）


In [42]:
import pandas as pd
import datetime
import numpy as np
import matplotlib.pyplot as plt
from sklearn import preprocessing
from sklearn.model_selection import train_test_split,cross_val_score
from sklearn import linear_model
%matplotlib inline
train_df = pd.read_csv('~/gci2017_team12/source_data/train.csv')
test_df = pd.read_csv("~/gci2017_team12/source_data/test.csv")

In [43]:
train_df_y = train_df[["y"]]
train_df = train_df.drop(["y"],axis=1)

In [65]:
train_df.head()

,datetime,client,close,price_am,price_pm,Month,DayofMonth,dayofweek,year,year_num,...,am_4,am_5,pm_-1,pm_0,pm_1,pm_2,pm_3,pm_4,pm_5,y
0,2010-07-01,0,0,-1,-1,7,1,3,2010,0,...,0,0,1,0,0,0,0,0,0,17
1,2010-07-02,0,0,-1,-1,7,2,4,2010,0,...,0,0,1,0,0,0,0,0,0,18
2,2010-07-03,0,0,-1,-1,7,3,5,2010,0,...,0,0,1,0,0,0,0,0,0,20
3,2010-07-04,0,0,-1,-1,7,4,6,2010,0,...,0,0,1,0,0,0,0,0,0,20
4,2010-07-05,0,0,-1,-1,7,5,0,2010,0,...,0,0,1,0,0,0,0,0,0,14


In [44]:
# train_df_y.reset_index()

In [45]:
X_df = pd.concat([train_df,test_df])
X_df['datetime'] = pd.to_datetime(X_df.datetime)
#何月か
X_df['Month'] = X_df['datetime'].dt.month
#何日か
X_df['DayofMonth'] = X_df['datetime'].dt.day
#何曜日か
X_df['dayofweek'] = X_df['datetime'].dt.dayofweek
#何年か
X_df['year'] = X_df['datetime'].dt.year
# 年をラベルづけ
le = preprocessing.LabelEncoder()
le.fit(X_df["year"])
X_df["year_num"] = le.transform(X_df["year"])

#datetimeを保存
dt_df = X_df[['datetime']]




In [46]:
#ダミー変数化
#何月か
month_df = pd.get_dummies(X_df['Month'], prefix='M', prefix_sep='_')
#何日か
dayofmonth_df = pd.get_dummies(X_df["DayofMonth"], prefix='DM', prefix_sep='_')
#何曜日か
dayofweek_df = pd.get_dummies(X_df['dayofweek'], prefix='DW', prefix_sep='_')

In [47]:
# priceダミー化
price_am_df = pd.get_dummies(X_df['price_am'], prefix="am", prefix_sep='_')
price_pm_df = pd.get_dummies(X_df['price_pm'], prefix="pm", prefix_sep='_')

In [48]:
price_am_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2466 entries, 0 to 364
Data columns (total 7 columns):
am_-1    2466 non-null uint8
am_0     2466 non-null uint8
am_1     2466 non-null uint8
am_2     2466 non-null uint8
am_3     2466 non-null uint8
am_4     2466 non-null uint8
am_5     2466 non-null uint8
dtypes: uint8(7)
memory usage: 36.1 KB


In [49]:
# 全部まるまるくっつける
X_df = pd.concat([X_df,
                month_df,
                dayofmonth_df,
                dayofweek_df, price_am_df, price_pm_df], axis=1)


In [50]:
train_df = X_df[0:2101].reset_index()
train_df = pd.concat([train_df,train_df_y],axis=1)
train_df = train_df.drop("index",axis=1)

In [51]:
train_df.

SyntaxError: invalid syntax (<ipython-input-51-a92e0778390e>, line 1)

In [52]:
test_df = X_df[2101:]

In [53]:
#closeが1なら0を返す必要性
pd.read_csv('~/gci2017_team12/source_data/train.csv').groupby("close")['y'].mean(),pd.read_csv('~/gci2017_team12/source_data/train.csv').groupby("close")["y"].size()

(close
 0    34.540501
 1     0.000000
 Name: y, dtype: float64, close
 0    2074
 1      27
 Name: y, dtype: int64)

In [66]:
train_df.columns

Index(['datetime', 'client', 'close', 'price_am', 'price_pm', 'Month',
       'DayofMonth', 'dayofweek', 'year', 'year_num', 'M_1', 'M_2', 'M_3',
       'M_4', 'M_5', 'M_6', 'M_7', 'M_8', 'M_9', 'M_10', 'M_11', 'M_12',
       'DM_1', 'DM_2', 'DM_3', 'DM_4', 'DM_5', 'DM_6', 'DM_7', 'DM_8', 'DM_9',
       'DM_10', 'DM_11', 'DM_12', 'DM_13', 'DM_14', 'DM_15', 'DM_16', 'DM_17',
       'DM_18', 'DM_19', 'DM_20', 'DM_21', 'DM_22', 'DM_23', 'DM_24', 'DM_25',
       'DM_26', 'DM_27', 'DM_28', 'DM_29', 'DM_30', 'DM_31', 'DW_0', 'DW_1',
       'DW_2', 'DW_3', 'DW_4', 'DW_5', 'DW_6', 'am_-1', 'am_0', 'am_1', 'am_2',
       'am_3', 'am_4', 'am_5', 'pm_-1', 'pm_0', 'pm_1', 'pm_2', 'pm_3', 'pm_4',
       'pm_5', 'y'],
      dtype='object')

In [84]:
# 居住率の辞書型を作ってforで回して入れていく
stay_rate_dic = {"stay_rate_itaku_kami": [90.3, 90.8, 89.2, 89.6, 90.1, 91.4, 90.2], "stay_rate_itaku_kouki": [92.0,  90.7, 90.3, 91.3, 91.5, 92.5, 95.0], "stay_rate_sublease_kami": [93.4, 94.4, 93.6, 92.2, 92.2, 92.0, 92.8], "stay_rate_sublease_kouki": [95.8, 93.4, 94.8, 93.9, 94.8, 95.3, 97.2]}
first_half = [4,5,6,7,8,9]
latter_half_1 = [10,11,12]
latter_half_2 = [1, 2, 3]


In [85]:
train_df["stay_rate_itaku"]=int(0)
train_df["stay_rate_sublease"]=int(0)

for a,b,c,d, year in zip(stay_rate_dic["stay_rate_itaku_kami"],
                stay_rate_dic["stay_rate_itaku_kouki"],
                stay_rate_dic["stay_rate_sublease_kami"],
               stay_rate_dic["stay_rate_sublease_kouki"],
                         [2010, 2011, 2012, 2013, 2014, 2015, 2016]):
    train_df["stay_rate_itaku"][(train_df["Month"].isin(first_half)) 
                                & (train_df["year"]==year)]= a
    train_df["stay_rate_sublease"][(train_df["Month"].isin(first_half)) 
                                & (train_df["year"]==year)]= a
    train_df["stay_rate_itaku"][(train_df["Month"].isin(latter_half_1)) 
                                & (train_df["year"]==year)]= b
    train_df["stay_rate_itaku"][(train_df["Month"].isin(latter_half_2)) 
                                & (train_df["year"] == year+1)]= b
    train_df["stay_rate_sublease"][(train_df["Month"].isin(latter_half_1)) 
                                & (train_df["year"] == year)]= d
    train_df["stay_rate_sublease"][(train_df["Month"].isin(latter_half_2)) 
                                & (train_df["year"] == year+1)]= d

/Users/nakatatoshinori/.pyenv/versions/3.6.1/lib/python3.6/site-packages/ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  # Remove the CWD from sys.path while we load stuff.
/Users/nakatatoshinori/.pyenv/versions/3.6.1/lib/python3.6/site-packages/ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if sys.path[0] == '':
/Users/nakatatoshinori/.pyenv/versions/3.6.1/lib/python3.6/site-packages/ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-

In [88]:
train_df.groupby("stay_rate_itaku").size()

stay_rate_itaku
89.2    183
89.6    183
90.1    183
90.3    274
90.7    183
90.8    183
91.3    182
91.4    183
91.5    182
92.0    182
92.5    183
dtype: int64

In [83]:
train_df[(train_df["Month"].isin(first_half)) & (train_df["year"]==2010)]["stay_rate_itaku"]

0     90.3
1     90.3
2     90.3
3     90.3
4     90.3
5     90.3
6     90.3
7     90.3
8     90.3
9     90.3
10    90.3
11    90.3
12    90.3
13    90.3
14    90.3
15    90.3
16    90.3
17    90.3
18    90.3
19    90.3
20    90.3
21    90.3
22    90.3
23    90.3
24    90.3
25    90.3
26    90.3
27    90.3
28    90.3
29    90.3
      ... 
62    90.3
63    90.3
64    90.3
65    90.3
66    90.3
67    90.3
68    90.3
69    90.3
70    90.3
71    90.3
72    90.3
73    90.3
74    90.3
75    90.3
76    90.3
77    90.3
78    90.3
79    90.3
80    90.3
81    90.3
82    90.3
83    90.3
84    90.3
85    90.3
86    90.3
87    90.3
88    90.3
89    90.3
90    90.3
91    90.3
Name: stay_rate_itaku, Length: 92, dtype: float64

In [55]:
#重回帰分析で予測
from sklearn import linear_model
clf = linear_model.LinearRegression()
 


# X = train_df.drop(["datetime",'dayofweek',"DayofMonth","Month","y","year"], axis=1)
# y = train_df["y"]

X = train_df[train_df["price_pm"] > -1].drop(["datetime",'dayofweek',"DayofMonth","Month","y","year"], axis=1)
Y = train_df[train_df["price_pm"] > -1].drop(["datetime",'dayofweek',"DayofMonth","Month", "year"], axis=1)["y"]
 
    

    
# # 予測モデルを作成
clf.fit(X, Y)
pred = clf.predict(X)
print(clf.score(X, y))
pred

0.727891336475


array([  5.79296875,   4.80859375,   4.51171875, ...,  87.6640625 ,
        85.90625   ,  88.95703125])

In [56]:
X.columns

Index(['client', 'close', 'price_am', 'price_pm', 'year_num', 'M_1', 'M_2',
       'M_3', 'M_4', 'M_5', 'M_6', 'M_7', 'M_8', 'M_9', 'M_10', 'M_11', 'M_12',
       'DM_1', 'DM_2', 'DM_3', 'DM_4', 'DM_5', 'DM_6', 'DM_7', 'DM_8', 'DM_9',
       'DM_10', 'DM_11', 'DM_12', 'DM_13', 'DM_14', 'DM_15', 'DM_16', 'DM_17',
       'DM_18', 'DM_19', 'DM_20', 'DM_21', 'DM_22', 'DM_23', 'DM_24', 'DM_25',
       'DM_26', 'DM_27', 'DM_28', 'DM_29', 'DM_30', 'DM_31', 'DW_0', 'DW_1',
       'DW_2', 'DW_3', 'DW_4', 'DW_5', 'DW_6', 'am_-1', 'am_0', 'am_1', 'am_2',
       'am_3', 'am_4', 'am_5', 'pm_-1', 'pm_0', 'pm_1', 'pm_2', 'pm_3', 'pm_4',
       'pm_5'],
      dtype='object')

In [57]:
X = train_df[train_df["price_pm"]>-1].drop(["datetime",'dayofweek',"DayofMonth","Month","y","year" , 'price_am', 'price_pm', 'close'], axis=1)
Y = train_df[train_df["price_pm"]>-1]["y"]
 
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.5, random_state=0)
# 年月日
# モデルのあてはめ
#モデルのインスタンス
l_model = linear_model.LinearRegression()
clf = l_model.fit(X_train,y_train)
print("train:",clf.__class__.__name__ , clf.score(X_train,y_train))
print("test:",clf.__class__.__name__ , clf.score(X_test,y_test))
 # 偏回帰係数
print(pd.DataFrame({"Name":X.columns,
"Coefficients":clf.coef_}).sort_values(by='Coefficients') )
# 切片
print(clf.intercept_)
# 交差検証法

# cv　で分割数を決定する。デフォルトは3。
scores = cross_val_score(clf,X_train,y_train,cv=5)

print("Cross validation scores:{}".format(scores))
print("Cross validation scores(mean):{}".format(scores.mean()))

train: LinearRegression 0.737360527575
test: LinearRegression 0.701555146288
    Coefficients      Name
19 -1.291705e+12      DM_6
20 -1.291705e+12      DM_7
21 -1.291705e+12      DM_8
24 -1.291705e+12     DM_11
18 -1.291705e+12      DM_5
16 -1.291705e+12      DM_3
27 -1.291705e+12     DM_14
17 -1.291705e+12      DM_4
26 -1.291705e+12     DM_13
22 -1.291705e+12      DM_9
23 -1.291705e+12     DM_10
30 -1.291705e+12     DM_17
25 -1.291705e+12     DM_12
15 -1.291705e+12      DM_2
32 -1.291705e+12     DM_19
31 -1.291705e+12     DM_18
29 -1.291705e+12     DM_16
34 -1.291705e+12     DM_21
28 -1.291705e+12     DM_15
14 -1.291705e+12      DM_1
36 -1.291705e+12     DM_23
35 -1.291705e+12     DM_22
38 -1.291705e+12     DM_25
33 -1.291705e+12     DM_20
37 -1.291705e+12     DM_24
40 -1.291705e+12     DM_27
42 -1.291705e+12     DM_29
39 -1.291705e+12     DM_26
44 -1.291705e+12     DM_31
41 -1.291705e+12     DM_28
..           ...       ...
12 -6.776484e+11      M_11
10 -6.776484e+11       M_9
11 -6

In [58]:
pred

array([  5.79296875,   4.80859375,   4.51171875, ...,  87.6640625 ,
        85.90625   ,  88.95703125])

## 居住率加えたver

同年のデータだとやや過学習ではあるが予測精度は上がった
前年のデータでやってみると、やはり過学習であるがデータがない時と比べれば精度が上がった。

In [90]:
# 同じ年ver 
# 居住率の辞書型を作ってforで回して入れていく
stay_rate_dic = {"stay_rate_itaku_kami": [90.3, 90.8, 89.2, 89.6, 90.1, 91.4, 90.2], "stay_rate_itaku_kouki": [92.0,  90.7, 90.3, 91.3, 91.5, 92.5, 95.0], "stay_rate_sublease_kami": [93.4, 94.4, 93.6, 92.2, 92.2, 92.0, 92.8], "stay_rate_sublease_kouki": [95.8, 93.4, 94.8, 93.9, 94.8, 95.3, 97.2]}
first_half = [4,5,6,7,8,9]
latter_half_1 = [10,11,12]
latter_half_2 = [1, 2, 3]
train_df["stay_rate_itaku"]=int(0)
train_df["stay_rate_sublease"]=int(0)

for a,b,c,d, year in zip(stay_rate_dic["stay_rate_itaku_kami"],
                stay_rate_dic["stay_rate_itaku_kouki"],
                stay_rate_dic["stay_rate_sublease_kami"],
               stay_rate_dic["stay_rate_sublease_kouki"],
                         [2010, 2011, 2012, 2013, 2014, 2015, 2016]):
    train_df["stay_rate_itaku"][(train_df["Month"].isin(first_half)) 
                                & (train_df["year"]==year)]= a
    train_df["stay_rate_sublease"][(train_df["Month"].isin(first_half)) 
                                & (train_df["year"]==year)]= a
    train_df["stay_rate_itaku"][(train_df["Month"].isin(latter_half_1)) 
                                & (train_df["year"]==year)]= b
    train_df["stay_rate_itaku"][(train_df["Month"].isin(latter_half_2)) 
                                & (train_df["year"] == year+1)]= b
    train_df["stay_rate_sublease"][(train_df["Month"].isin(latter_half_1)) 
                                & (train_df["year"] == year)]= d
    train_df["stay_rate_sublease"][(train_df["Month"].isin(latter_half_2)) 
                                & (train_df["year"] == year+1)]= d

/Users/nakatatoshinori/.pyenv/versions/3.6.1/lib/python3.6/site-packages/ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app
/Users/nakatatoshinori/.pyenv/versions/3.6.1/lib/python3.6/site-packages/ipykernel_launcher.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Users/nakatatoshinori/.pyenv/versions/3.6.1/lib/python3.6/site-packages/ipykernel_launcher.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Users/nakatatoshino

In [93]:
X = train_df[train_df["price_pm"]>-1].drop(["datetime",'dayofweek',"DayofMonth","Month","y","year" , 'price_am', 'price_pm', 'close'], axis=1)
Y = train_df[train_df["price_pm"]>-1]["y"]

X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.5, random_state=0)
# 年月日
# モデルのあてはめ
#モデルのインスタンス
l_model = linear_model.LinearRegression()
clf = l_model.fit(X_train,y_train)
print("train:",clf.__class__.__name__ , clf.score(X_train,y_train))
print("test:",clf.__class__.__name__ , clf.score(X_test,y_test))
 # 偏回帰係数
print(pd.DataFrame({"Name":X.columns,
"Coefficients":clf.coef_}).sort_values(by='Coefficients') )
# 切片
print(clf.intercept_)
# 交差検証法

# cv　で分割数を決定する。デフォルトは3。
scores = cross_val_score(clf,X_train,y_train,cv=5)

print("Cross validation scores:{}".format(scores))
print("Cross validation scores(mean):{}".format(scores.mean()))
 

train: LinearRegression 0.776930668582
test: LinearRegression 0.728246522935
    Coefficients                Name
53     -8.527688                am_0
19     -6.569195                DM_6
2      -6.526551                 M_1
66     -6.504607     stay_rate_itaku
21     -6.274303                DM_8
20     -5.872057                DM_7
16     -4.589602                DM_3
24     -4.244934               DM_11
27     -4.053186               DM_14
26     -3.970066               DM_13
17     -3.907210                DM_4
18     -3.863407                DM_5
54     -3.589693                am_1
23     -3.254230               DM_10
6      -3.190220                 M_5
22     -3.134369                DM_9
47     -2.907672                DW_2
60     -2.756933                pm_0
8      -2.503381                 M_7
30     -2.487624               DM_17
7      -2.265805                 M_6
15     -2.255109                DM_2
25     -2.027374               DM_12
12     -1.970668                M_1

In [94]:
# 一年前ver
# 居住率の辞書型を作ってforで回して入れていく
stay_rate_dic = {"stay_rate_itaku_kami": [90.6, 90.3, 90.8, 89.2, 89.6, 90.1, 91.4], "stay_rate_itaku_kouki": [91.4, 92.0,  90.7, 90.3, 91.3, 91.5, 92.5], "stay_rate_sublease_kami": [93.2, 93.4, 94.4, 93.6, 92.2, 92.2, 92.0], "stay_rate_sublease_kouki": [94.4, 95.8, 93.4, 94.8, 93.9, 94.8, 95.3]}
first_half = [4,5,6,7,8,9]
latter_half_1 = [10,11,12]
latter_half_2 = [1, 2, 3]
train_df["stay_rate_itaku"]=int(0)
train_df["stay_rate_sublease"]=int(0)

for a,b,c,d, year in zip(stay_rate_dic["stay_rate_itaku_kami"],
                stay_rate_dic["stay_rate_itaku_kouki"],
                stay_rate_dic["stay_rate_sublease_kami"],
               stay_rate_dic["stay_rate_sublease_kouki"],
                         [2010, 2011, 2012, 2013, 2014, 2015, 2016]):
    train_df["stay_rate_itaku"][(train_df["Month"].isin(first_half)) 
                                & (train_df["year"]==year)]= a
    train_df["stay_rate_sublease"][(train_df["Month"].isin(first_half)) 
                                & (train_df["year"]==year)]= a
    train_df["stay_rate_itaku"][(train_df["Month"].isin(latter_half_1)) 
                                & (train_df["year"]==year)]= b
    train_df["stay_rate_itaku"][(train_df["Month"].isin(latter_half_2)) 
                                & (train_df["year"] == year+1)]= b
    train_df["stay_rate_sublease"][(train_df["Month"].isin(latter_half_1)) 
                                & (train_df["year"] == year)]= d
    train_df["stay_rate_sublease"][(train_df["Month"].isin(latter_half_2)) 
                                & (train_df["year"] == year+1)]= d

/Users/nakatatoshinori/.pyenv/versions/3.6.1/lib/python3.6/site-packages/ipykernel_launcher.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  app.launch_new_instance()
/Users/nakatatoshinori/.pyenv/versions/3.6.1/lib/python3.6/site-packages/ipykernel_launcher.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Users/nakatatoshinori/.pyenv/versions/3.6.1/lib/python3.6/site-packages/ipykernel_launcher.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Users/nakatatoshinori/.pyenv/ver

In [95]:
X = train_df[train_df["price_pm"]>-1].drop(["datetime",'dayofweek',"DayofMonth","Month","y","year" , 'price_am', 'price_pm', 'close'], axis=1)
Y = train_df[train_df["price_pm"]>-1]["y"]

X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.5, random_state=0)
# 年月日
# モデルのあてはめ
#モデルのインスタンス
l_model = linear_model.LinearRegression()
clf = l_model.fit(X_train,y_train)
print("train:",clf.__class__.__name__ , clf.score(X_train,y_train))
print("test:",clf.__class__.__name__ , clf.score(X_test,y_test))
 # 偏回帰係数
print(pd.DataFrame({"Name":X.columns,
"Coefficients":clf.coef_}).sort_values(by='Coefficients') )
# 切片
print(clf.intercept_)
# 交差検証法

# cv　で分割数を決定する。デフォルトは3。
scores = cross_val_score(clf,X_train,y_train,cv=5)

print("Cross validation scores:{}".format(scores))
print("Cross validation scores(mean):{}".format(scores.mean()))
 

train: LinearRegression 0.758608761169
test: LinearRegression 0.721202719243
    Coefficients                Name
6      -9.404132                 M_5
7      -8.494637                 M_6
8      -8.174656                 M_7
53     -8.050571                am_0
9      -6.813387                 M_8
19     -6.293249                DM_6
20     -5.985624                DM_7
21     -5.415345                DM_8
10     -5.217068                 M_9
24     -4.980139               DM_11
18     -4.713493                DM_5
16     -4.152532                DM_3
17     -4.096519                DM_4
26     -3.883897               DM_13
27     -3.829426               DM_14
22     -3.485668                DM_9
66     -3.375477     stay_rate_itaku
47     -3.284908                DW_2
23     -3.274701               DM_10
54     -3.189905                am_1
30     -2.834015               DM_17
60     -2.832790                pm_0
25     -2.622919               DM_12
46     -2.054945                DW_

In [89]:
# #xgboostで回帰モデル生成
# import xgboost as xgb
# #モデルのインスタンス作成
# mod = xgb.XGBRegressor()
# #欠損値のデータを学習させずにモデル生成
# X = train_df[train_df["price_pm"]>-1].drop(["datetime",'dayofweek',"DayofMonth","Month","y","year" , 'price_am', 'price_pm'], axis=1)
# Y = train_df[train_df["price_pm"]>-1]["y"]
# mod.fit(X,Y)

# pred_xgb= mod.predict(test_df.drop(["datetime",'dayofweek',"DayofMonth","Month","y","year" , 'price_am', 'price_pm'], axis=1))

# test_df["y"] = pred_xgb
# test_df["y"][test_df["close"]==1] = int(0)
# submission = pd.concat([test_df.datetime, test_df.y], axis=1)
# submission.to_csv("submission.csv", index=False,header=False)

In [38]:
# # 決定木
# from sklearn import tree
# from sklearn.tree import  DecisionTreeClassifier

# X = train_df[train_df["price_pm"]>-1].drop(["datetime",'dayofweek',"DayofMonth","Month","y","year" , 'price_am', 'price_pm', 'close'], axis=1)
# Y = train_df[train_df["price_pm"]>-1]["y"]
 
# X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.5, random_state=0)
# # 年月日

# # 決定木インスタンス（エントロピー、深さ5）
# tree_model = DecisionTreeClassifier(criterion='entropy',max_depth=5, random_state=50)

# tree_model.fit(X_train,y_train)

# print("train:",tree_model.__class__.__name__ ,tree_model.score(X_train,y_train))
# print("test:",tree_model.__class__.__name__ , tree_model.score(X_test,y_test))


# # cv　で分割数を決定する。デフォルトは3。
# scores = cross_val_score(tree_model,X_train,y_train,cv=5)

# print("Cross validation scores:{}".format(scores))
# print("Cross validation scores(mean):{}".format(scores.mean()))

train: DecisionTreeClassifier 0.128177966102
test: DecisionTreeClassifier 0.0349206349206
Cross validation scores:[ 0.01731602  0.05050505  0.05945946  0.04705882  0.05625   ]
Cross validation scores(mean):0.04611787016198781


/Users/nakatatoshinori/.pyenv/versions/3.6.1/lib/python3.6/site-packages/sklearn/model_selection/_split.py:581: Warning: The least populated class in y has only 1 members, which is too few. The minimum number of groups for any class cannot be less than n_splits=5.
  % (min_groups, self.n_splits)), Warning)


In [ ]:
submission = pd.concat([test_df.datetime, test_df.y], axis=1)
submission.to_csv("submission.csv", index=False,header=False)